# Взаимодействие с треками

Для взаимодействия с треками вам **необходим** корректный токен.

Получаем токен из окружения среды

In [1]:
val yamToken = System.getenv("YAM_TOKEN")!!

Инициализируем клиента

In [2]:
import su.pank.yamapi.builder.createYaMusicApiClient

val client = createYaMusicApiClient {
    token = yamToken
}

## Получение трека по его ID

Получение трека https://music.yandex.ru/album/22049037/track/728014

In [3]:
import kotlinx.coroutines.runBlocking

val track = runBlocking {
     client.tracks("728014").first()
}
track

Track(id=728014, title=La Maritza, available=true, availableForPremiumUsers=true, availableFullWithoutPermission=false, availableForOptions=[Bookmate], durationMs=220180, previewDurationMs=30000, storageDir=51327_109b74ca.36526310.1.609676, fileSize=0, r128=R128(i=-9.74, tp=0.7), artists=[Artist(id=18354, name=Sylvie Vartan, cover=Cover(type=FromAlbumCover, uri=avatars.yandex.net/get-music-content/6300975/8c474d84.a.22048939-1/%%, prefix=8c474d84.a.22048939-1, dir=null, version=null, custom=null), various=false, composer=false, genres=[])], albums=[Album(id=22048939, title='Triple Best Of', type=null, metaType=Music, year=2009, releaseDate=2009-11-16T21:00:00Z, coverUri=avatars.yandex.net/get-music-content/6300975/8c474d84.a.22048939-1/%%, ogImage='avatars.yandex.net/get-music-content/6300975/8c474d84.a.22048939-1/%%', genre=pop, trackCount=60, likesCount=154, recent=false, veryImportant=false, artists=[Artist(id=18354, name=Sylvie Vartan, cover=Cover(type=FromAlbumCover, uri=avatars.y

Как вы могли заметить, метод `tracks` принимает на вход **список** ID треков и возвращает **список** треков.

In [4]:
val tracks = runBlocking {
    client.tracks("62690680", "728014")
}
tracks.forEach {
    println(it.title + " - " + it.artists.joinToString(", ") { artist -> artist.name })
}

Voyage Voyage - Meimuna
La Maritza - Sylvie Vartan


## Скачивание трека

> Внимание! Скачивание треков может нарушать авторские права. Убедитесь, что у вас есть право на скачивание и использование треков в соответствии с законодательством вашей страны и условиями использования сервиса Яндекс Музыка.

Для скачивания трека нам понадобится его `downloadInfo`, который мы можем получить из объекта трека.

In [5]:
val downloadInfos = runBlocking { track.downloadInfo() }

downloadInfos.forEach { info ->
    println("${info.codec}-${info.bitrateInKbps}")
}

MP3-192
MP3-320


Как можете заметить, мы получаем в различных форматах и битрейтах.

Выбираем определённый формат и скачиваем:

In [6]:
import su.pank.yamapi.downloadInfo.model.Codec

val selected = downloadInfos.first { it.codec == Codec.MP3 && it.bitrateInKbps == 320  }
runBlocking {
    selected.download().take(10)
}

[-1, -5, -32, 68, 0, 3, 54, 51, -124, -68]

**Замечание:** `DownloadInfo` долго не живут (примерно 5 минут), так что получайте `DownloadInfo` перед самим скачиванием трека.

### Получние flac

In [ ]:
runBlocking {
    client.tracks.losslessInfo(track.id)
}

## Текст песни

In [10]:
runBlocking {
    track.lyrics().fetch()
}

La Maritza, c'est ma rivière
Comme la Seine est la tienne
Mais il n'y a que mon père
Maintenant qui s'en souvienne
Quelquefois

De mes dix premières années
Il ne me reste plus rien
Pas la plus pauvre poupée
Plus rien qu'un petit refrain
D'autrefois

Tous les oiseaux de ma rivière
Nous chantaient la liberté
Moi je ne comprenais guère
Mais mon père, lui, savait
Écouter

Quand l'horizon s'est fait trop noir
Tous les oiseaux sont partis
Sur les chemins de l'espoir
Et nous on les a suivis
À Paris

De mes dix premières années
Il ne me reste plus rien, rien
Et pourtant les yeux fermés
Moi j'entends mon père chanter
Ce refrain

## Текс песни с временными метками

In [11]:
import su.pank.yamapi.track.model.lyrics.LyricsFormat

runBlocking {
    track.lyrics(LyricsFormat.LRC).fetch()
}

[00:29.33] La Maritza, c'est ma rivière
[00:34.46] Comme la Seine est la tienne
[00:37.61] Mais il n'y a que mon père
[00:41.19] Maintenant qui s'en souvienne
[00:46.25] Quelquefois
[00:50.79] De mes dix premières années
[00:55.90] Il ne me reste plus rien
[00:59.35] Pas la plus pauvre poupée
[01:02.84] Plus rien qu'un petit refrain
[01:08.14] D'autrefois
[01:51.44] Tous les oiseaux de ma rivière
[01:56.05] Nous chantaient la liberté
[01:59.48] Moi je ne comprenais guère
[02:02.44] Mais mon père, lui, savait
[02:08.31] Écouter
[02:13.18] Quand l'horizon s'est fait trop noir
[02:17.63] Tous les oiseaux sont partis
[02:20.82] Sur les chemins de l'espoir
[02:24.34] Et nous on les a suivis
[02:29.47] À Paris
[02:35.57] De mes dix premières années
[02:37.99] Il ne me reste plus rien, rien
[02:43.38] Et pourtant les yeux fermés
[02:46.32] Moi j'entends mon père chanter
[02:51.42] Ce refrain
[03:38.49] 

## Обложки трека

In [9]:
import su.pank.yamapi.model.cover.CoverSize

CoverSize.entries.joinToString("\n") {
    "<img src='${track.urlCover(it)}' alt='cover'/><p>${it.name}</p>"
}.let { HTML(it) }

50x50 
 100x100 
 200x200 
 400x400 
 600x600 
 800x800 
 1000x1000